In [ ]:
import math
import numpy as np
from scipy.fft import fft, fftfreq
import matplotlib.pyplot as plt
import seaborn as sns

import sys
if '..' not in sys.path:
    sys.path = ['..'] + sys.path
from multitone import *

This notebook implements the algorithm described in the following paper:

S. Boyd, "Multitone signals with low crest factor," in _IEEE Transactions on Circuits and Systems_, vol. 33, no. 10, pp. 1018-1022, October 1986, doi: 10.1109/TCS.1986.1085837.

Compute the time series and crest factors for the first 100 tones:

In [ ]:
N_tones = np.arange(1, 101)
F = N_tones[0] / (2 * np.pi)
T = 1 / F
dt = T / 1000
fs = 1 / dt
t = np.r_[0 : 5 * T : dt]
tend = t[-1]
U = {N: multitone(N, fs, tend, method='N') for N in N_tones}
t = np.arange(U[N_tones[0]].size) / fs
CF = {N: compute_crest_factor(u, 1 / fs) for N, u in U.items()}

Plot the crest factor:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5,3))
ax.plot(N_tones, 20 * np.log10(list(CF.values())))
ax.set_xlabel('No. of tones')
ax.set_ylabel('Crest factor [dB]')
sns.despine()
fig.tight_layout()

Plot the time series, its distribution and Fourier spectra for a number of tones equal to 32.

In [ ]:
N = 32
u = U[N]
print('Crest factor for no. of tones = {}: {:g} ({:g} dB).'.format(N, CF[N], 20 * np.log10(CF[N])))
n, x = compute_amplitude_distribution(u, bins='fd')

uf = fft(u - u.mean())
ups = 2 / u.size * np.abs(uf[:u.size // 2])
freq = fftfreq(u.size, 1 / fs)[:u.size//2]
ω = 2 * np.pi * freq

fig,ax = plt.subplots(3, 1, figsize=(5,5))
ax[0].plot(t, u, lw=0.75)
ax[0].set_xlabel('Time [s]')
ax[0].set_ylabel('u')
ax[0].set_xlim([0, 10])
ax[0].set_ylim([-2.5, 2.5])

ax[1].plot(x, n, 'k', lw=0.75)
ax[1].set_xlim([0, 2])
ax[1].set_ylim([0, 1])
ax[1].set_xlabel('Amplitude')
ax[1].set_ylabel(r'$F_u(a)$')

ax[2].vlines(ω, 0 * ups, ups, lw=0.5)
ax[2].hlines(np.sqrt(2 / N), 0, N + 1, color='tab:red', ls='--', lw=1)
ax[2].plot(ω, ups, 'ko', markerfacecolor='w', markersize=3, lw=0.75)
ax[2].set_xlim([0, N + 1.1])
ax[2].set_xlabel('ω (rad/s)')
ax[2].set_ylabel('|FFT(u)|')

sns.despine()
fig.tight_layout()